### Engineering

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

In [4]:
import torch.distributed as dist
import torch.distributed.rpc as rpc

In [5]:
from einops import rearrange

In [ ]:
inputs = rearrange(
    inputs,
    "batch_size seq_len hidden_size -> (batch_size seq_len) hidden_size"
)

In [ ]:
probs = F.softmax(inputs, dim=-1)

In [ ]:
_, idxs = torch.max(probs, dim=-1)

In [ ]:
class P2P:
    def _send_metadata(self, data, dst_rank, parallel_context, parallel_mode):
        process_group = parallel_context.get_group(parallel_mode)
        
        dtype = torch.tensor(DTYPE_TO_ID[data.dtype])
        dist.send(dtype, dst=dst_rank, group=process_group)
        
        requires_grad = torch.tensor(1 if data.requires_grad else 0)
        dist.send(requires_grad, dst=dst_rank, group=process_group)
        
        shape = torch.tensor
    
    def send(self, data, dst_rank, parallel_context, parallel_mode):
        process_group = parallel_context.get_group(parallel_mode)
        self._send_metadata(data, dst_rank, parallel_context, parallel_mode)
        dist.send(data, dst=dst_rank, group=process_group)

In [ ]:
def send(data, src_rank, dst_rank, parallel_context, parallel_mode = ParallelMode.PIPELINE):
    rank = parallel_context.get_local_rank(parallel_mode)
    if rank == src_rank:
        P2P().send(data, dst_rank, parallel_context, parallel_mode)

In [ ]:
running, ready, failed, succeed, cooldown, block